In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
from tqdm import tqdm


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


print("\nSelect Dataset:")
print("  1 → MNIST")
print("  2 → Fashion-MNIST")

dataset_option = input("Enter choice (1 or 2): ").strip()
epochs = int(input("Enter number of epochs (30-100): "))
batch_size = int(input("Enter batch size (64 or 128): "))
noise_dim = int(input("Enter noise dimension (50 or 100): "))
learning_rate = float(input("Enter learning rate (e.g., 0.0002): "))
save_interval = int(input("Enter save interval (e.g., 5): "))


# Data Transform

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


# Dataset Loading

if dataset_option == "1":
    dataset = torchvision.datasets.MNIST(
        root="./data", train=True, download=True, transform=transform
    )
    class_names = [str(i) for i in range(10)]
    img_channels = 1
    print("✅ Using MNIST dataset")

elif dataset_option == "2":
    dataset = torchvision.datasets.FashionMNIST(
        root="./data", train=True, download=True, transform=transform
    )
    class_names = [
        "T-shirt", "Trouser", "Pullover", "Dress", "Coat",
        "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
    ]
    img_channels = 1
    print("✅ Using Fashion-MNIST dataset")

else:
    raise ValueError("❌ Invalid dataset choice")

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


# Generator

class Generator(nn.Module):
    def __init__(self, noise_dim, img_channels):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(noise_dim, 128 * 7 * 7),
            nn.ReLU(True),
            nn.Unflatten(1, (128, 7, 7)),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)


# Discriminator

class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(img_channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 1, 7, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x).view(-1)


# Initialize Models

G = Generator(noise_dim, img_channels).to(device)
D = Discriminator(img_channels).to(device)

def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

G.apply(weights_init)
D.apply(weights_init)


# Loss & Optimizers

criterion = nn.BCELoss()
optimizerG = optim.Adam(G.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizerD = optim.Adam(D.parameters(), lr=learning_rate, betas=(0.5, 0.999))

fixed_noise = torch.randn(25, noise_dim, device=device)


# Output Directories

os.makedirs("generated_samples", exist_ok=True)
os.makedirs("final_generated_images", exist_ok=True)


# Training Loop

print("\n🚀 Starting GAN Training...")

for epoch in range(epochs):
    d_loss_total = 0
    g_loss_total = 0

    for real_imgs, _ in tqdm(dataloader):
        real_imgs = real_imgs.to(device)
        batch_size_curr = real_imgs.size(0)

        # -------- Train Discriminator --------
        optimizerD.zero_grad()

        real_labels = torch.ones(batch_size_curr, device=device)
        fake_labels = torch.zeros(batch_size_curr, device=device)

        real_output = D(real_imgs)
        d_real_loss = criterion(real_output, real_labels)

        noise = torch.randn(batch_size_curr, noise_dim, device=device)
        fake_imgs = G(noise)
        fake_output = D(fake_imgs.detach())
        d_fake_loss = criterion(fake_output, fake_labels)

        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        optimizerD.step()

        # -------- Train Generator --------
        optimizerG.zero_grad()
        fake_output = D(fake_imgs)
        g_loss = criterion(fake_output, real_labels)
        g_loss.backward()
        optimizerG.step()

        d_loss_total += d_loss.item()
        g_loss_total += g_loss.item()

    # -------- Logging & Saving --------
    if (epoch + 1) % save_interval == 0:
        print(
            f"Epoch [{epoch+1}/{epochs}] | "
            f"D Loss: {d_loss_total/len(dataloader):.3f} | "
            f"G Loss: {g_loss_total/len(dataloader):.3f}"
        )

        with torch.no_grad():
            fake_samples = G(fixed_noise)
            vutils.save_image(
                fake_samples,
                f"generated_samples/epoch_{epoch+1:03d}.png",
                normalize=True,
                nrow=5
            )

# Final Image Generation

print("\n🎉 Training completed! Generating final images...")

with torch.no_grad():
    final_noise = torch.randn(100, noise_dim, device=device)
    final_imgs = G(final_noise).cpu()
    vutils.save_image(
        final_imgs,
        "final_generated_images/final_100.png",
        normalize=True,
        nrow=10
    )

print("\n✅ Done!")
print("📁 generated_samples/ → epoch-wise images")
print("📁 final_generated_images/final_100.png")


Using device: cuda

Select Dataset:
  1 → MNIST
  2 → Fashion-MNIST
Enter choice (1 or 2): 1
Enter number of epochs (30-100): 55
Enter batch size (64 or 128): 64
Enter noise dimension (50 or 100): 100
Enter learning rate (e.g., 0.0002): 0.0002
Enter save interval (e.g., 5): 5


100%|██████████| 9.91M/9.91M [00:01<00:00, 5.03MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.86MB/s]


✅ Using MNIST dataset

🚀 Starting GAN Training...


100%|██████████| 938/938 [00:19<00:00, 48.34it/s]


Epoch [5/55] | D Loss: 0.702 | G Loss: 1.891


...

Epoch [45/55] | D Loss: 0.466 | G Loss: 2.945


100%|██████████| 938/938 [00:19<00:00, 48.81it/s]


Epoch [50/55] | D Loss: 0.477 | G Loss: 2.986


100%|██████████| 938/938 [00:19<00:00, 48.78it/s]

Epoch [55/55] | D Loss: 0.462 | G Loss: 3.074

🎉 Training completed! Generating final images...

✅ Done!
📁 generated_samples/ → epoch-wise images
📁 final_generated_images/final_100.png
